In [1]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP
from selenium import webdriver
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [2]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210305
dia = 05
mes = 03
año = 2021
05/03/2021


In [3]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [4]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']

# ------------------------------------------------------------------------

# DOGC

In [5]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [6]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8357']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Anuncios_de_la_Administración_de_justicia_']


In [7]:
#response

## Recoge Nombre Secciones Sumario

In [8]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [9]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia


## Recoge Valores para formar URLs Secciones Sumario

In [10]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Anuncios_de_la_Administración_de_justicia_


In [11]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [12]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [13]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [14]:
df_sumarios['URL_Seccion'][0]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Disposiciones'

## Recoge Items en Seccion Disposiciones

In [15]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [16]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [17]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Disposiciones


In [18]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [19]:
URL_HTML_disposiciones

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8357#section-Disposiciones'

In [20]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de Acción Exterior, Relaciones Institucionales y Transparencia', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Cultura', 'Departamento de Justicia', 'Departamento de Agricultura, Ganadería, Pesca y Alimentación']
['RESOLUCIÓN EXI/583/2021, de 24 de febrero, por la que se da publicidad al Convenio de cooperación educativa entre la Facultad de Economía y Empresa de la Universidad de Barcelona (UB) y la Agencia Catalana de Cooperación al Desarrollo (ACCD) para la realización de prácticas académicas externas.', 'EDICTO de 25 de febrero de 2021, sobre acuerdos de la Comisión Territorial de Urbanismo de la Cataluña Central referentes al municipio de Sallent.', 'EDICTO de 26 de febrero de 2021, sobre un acuerdo de la Comisión de Territorio de Cataluña referente al municipio de Vilanova del Vallès.', 'EDICTO de 26 de febrero de 2021, sobre un acuerdo de la Comisión Territorial de Urbanismo del ámbito metropolitano de Barcelona y una resolución del secretario de

In [21]:
len(departamentos)

5

In [22]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [23]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [24]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [25]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,"RESOLUCIÓN EXI/583/2021, de 24 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"EDICTO de 25 de febrero de 2021, sobre acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"EDICTO de 26 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"EDICTO de 26 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"RESOLUCIÓN TES/585/2021, de 23 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,"RESOLUCIÓN CLT/592/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"RESOLUCIÓN CLT/576/2021, de 1 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,"RESOLUCIÓN CLT/577/2021, de 1 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,"RESOLUCIÓN CLT/589/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,"RESOLUCIÓN CLT/590/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [26]:
DOGC_sumarios['item_urlHTML'][1]

'https://dogc.gencat.cat/es/document-del-dogc/?documentId=894866'

In [27]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(3)

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

In [28]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC
0,"RESOLUCIÓN EXI/583/2021, de 24 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,En fecha 1 de febrero de 2021 se ha firmado el...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[24/02/2021],[EXI/0583/2021],[21062037],[Agencia Catalana de Cooperación al Desarrollo],[CVE-DOGC-B-21062037-2021],[8357],[05/03/2021],[Disposiciones]
1,"EDICTO de 25 de febrero de 2021, sobre acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de la Cat...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[25/02/2021],[],[21057021],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21057021-2021],[8357],[05/03/2021],[Disposiciones]
2,"EDICTO de 26 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"La Comisión de Territorio de Cataluña, en la s...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[26/02/2021],[],[21057014],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21057014-2021],[8357],[05/03/2021],[Disposiciones]
3,"EDICTO de 26 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo del ámbit...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[26/02/2021],[],[21057074],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21057074-2021],[8357],[05/03/2021],[Disposiciones]
4,"RESOLUCIÓN TES/585/2021, de 23 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El artículo 3.6 del Decreto 1/2018, de 19 de m...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[23/02/2021],[TES/0585/2021],[21062039],[Agencia de la Vivienda de Cataluña],[CVE-DOGC-B-21062039-2021],[8357],[05/03/2021],[Disposiciones]
5,"RESOLUCIÓN CLT/592/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Considerando el Acuerdo del Consejo de Adminis...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[CLT/0592/2021],[21062040],[Oficina de Apoyo a la Iniciativa Cultural],[CVE-DOGC-B-21062040-2021],[8357],[05/03/2021],[Disposiciones]
6,"RESOLUCIÓN CLT/576/2021, de 1 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,\nVisto el Acuerdo del Consejo de Administrac...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[01/03/2021],[CLT/0576/2021],[21061067],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21061067-2021],[8357],[05/03/2021],[Disposiciones]
7,"RESOLUCIÓN CLT/577/2021, de 1 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[01/03/2021],[CLT/0577/2021],[21061068],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21061068-2021],[8357],[05/03/2021],[Disposiciones]
8,"RESOLUCIÓN CLT/589/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[CLT/0589/2021],[21062044],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21062044-2021],[8357],[05/03/2021],[Disposiciones]
9,"RESOLUCIÓN CLT/590/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADO

In [29]:
DOGC_sumarios['item_fullText'][0]

'En fecha 1 de febrero de 2021 se ha firmado el Convenio de cooperación educativa entre la Facultad de Economía y Empresa de la Universidad de Barcelona (UB) y la Agencia Catalana de Cooperación al Desarrollo (ACCD) para la realización de prácticas académicas externas.\nVistos el artículo 112.2 de la Ley 26/2010, de 3 de agosto, de régimen jurídico y de procedimiento de las administraciones públicas de Cataluña, y el artículo 11 de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público,\n\xa0\nResuelvo:\n\xa0\nHacer público el convenio de cooperación educativa entra la Facultad de Economía y Empresa de la Universidad de Barcelona (UB) y la Agencia Catalana de Cooperación al Desarrollo (ACCD) para la realización de prácticas académicas externas.\n \xa0\nBarcelona, 24 de febrero de 2021\n\xa0\nCarme Gual Via\nDirectora de la Agencia Catalana de Cooperación al Desarrollo\n\xa0\n\xa0\nConvenio de cooperación educativa entra la Facultad de Economía y Empresa de la Universid

In [30]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [31]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [32]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

['Ley 26/2010', 'Ley 40/2015', 'Resolución EXI/1349/2017', 'Real Decreto 592/2014', 'Real Decreto 592/2014', 'Real Decreto 592/2014', 'Real Decreto 1493/2011', 'Ley 27/2011', 'ley 8/2014', 'Reglamento (UE) 2016/679', 'Real Decreto 592/2014', 'Real Decreto 592/2014', 'Real Decreto 592/2014', 'Ley 19/2014']
['Ley 3/2012', 'Real decreto 463/2020', 'Real decreto 465/2020', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 5/2020', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Resolución de 18 de enero de 2021', 'Ley 3/2012', 'Ley 3/2012', 'Ley 3/2012', 'Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 13/2009', 'Resolución GAH/380/2016', 'Resolución TES/662/2020', 'Ley 38/2003', 'Real decreto 887/2006', 'Ley 19/2014', 'Resolución TES/662/2020', 'Resolución TES/662/2020', 'Resolución TES/662/2020', 'Resolución GAH/380/2016', 'Ley 26/2010', 'Ley 26/20

In [33]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

['Ley 19/2014', 'Resolución EXI/1349/2017', 'ley 8/2014', 'Reglamento (UE) 2016/679', 'Real Decreto 592/2014', 'Ley 27/2011', 'Ley 40/2015', 'Real Decreto 1493/2011', 'Ley 26/2010']
['Ley 29/1998', 'Real decreto 463/2020', 'Ley 3/2012', 'Ley 2/2007', 'Real decreto 465/2020', 'Ley 39/2015']
['Ley 39/2015', 'Ley 29/1998', 'Ley 5/2020', 'Ley 3/2012', 'Ley 2/2007']
['Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Resolución de 18 de enero de 2021', 'Ley 2/2007']
['Ley 38/2003', 'Ley 19/2014', 'Ley 13/2009', 'Ley 39/2015', 'Ley 29/1998', 'Resolución GAH/380/2016', 'Ley 26/2010', 'Real decreto 887/2006', 'Resolución TES/662/2020']
['Ley 38/2003', 'Resolución CLT/198/2021', 'Ley 9/1993', 'Real decreto 887/2006', 'Resolución CLT/3216/2020']
['Resolución CLT/187/2021', 'Ley 38/2003', 'Ley 17/2015', 'Resolución CLT/3142/2020']
['Ley 38/2003', 'Ley 17/2015', 'Ley 20/2010', 'Reglamento (UE) 1407/2013', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020']
['Resolución CLT/187/2021', 'Ley 38/2003', '

In [34]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [35]:
print(regex_result)

['Ley 19/2014', 'Resolución EXI/1349/2017', 'ley 8/2014', 'Reglamento (UE) 2016/679', 'Real Decreto 592/2014', 'Ley 27/2011', 'Ley 40/2015', 'Real Decreto 1493/2011', 'Ley 26/2010', 'Ley 29/1998', 'Real decreto 463/2020', 'Ley 3/2012', 'Ley 2/2007', 'Real decreto 465/2020', 'Ley 39/2015', 'Ley 39/2015', 'Ley 29/1998', 'Ley 5/2020', 'Ley 3/2012', 'Ley 2/2007', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Resolución de 18 de enero de 2021', 'Ley 2/2007', 'Ley 38/2003', 'Ley 19/2014', 'Ley 13/2009', 'Ley 39/2015', 'Ley 29/1998', 'Resolución GAH/380/2016', 'Ley 26/2010', 'Real decreto 887/2006', 'Resolución TES/662/2020', 'Ley 38/2003', 'Resolución CLT/198/2021', 'Ley 9/1993', 'Real decreto 887/2006', 'Resolución CLT/3216/2020', 'Resolución CLT/187/2021', 'Ley 38/2003', 'Ley 17/2015', 'Resolución CLT/3142/2020', 'Ley 38/2003', 'Ley 17/2015', 'Ley 20/2010', 'Reglamento (UE) 1407/2013', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Resolución CLT/187/2021', 'Ley 38/2003', 'Resoluci

In [36]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [37]:
boletin_flat_list

['Ley 19/2014',
 'Resolución EXI/1349/2017',
 'Ley 18/1990',
 'Resolución CLT/336/2015',
 'Real Decreto 592/2014',
 'Resolución de 22 de febrero de 2021',
 'Ley 3/2012',
 'Reglamento (UE) 1407/2013',
 'Real decreto 887/2006',
 'Real Decreto 1493/2011',
 'Real decreto 465/2020',
 'Ley 39/2015',
 'Ley 2/2007',
 'Ley 26/2010',
 'Resolución CLT/3142/2020',
 'Ley 13/2009',
 'Resolución CLT/1621/2017',
 'ley 8/2014',
 'Ley 29/1998',
 'Ley 5/2020',
 'Resolución CLT/1645/2016',
 'Real decreto 463/2020',
 'Ley 27/2011',
 'Ley 9/1993',
 'Ley 40/2015',
 'Resolución CLT/1495/2018',
 'Resolución CLT/187/2021',
 'Resolución CLT/2991/2015',
 'Resolución de 18 de enero de 2021',
 'Resolución CLT/3216/2020',
 'Resolución TES/662/2020',
 'Resolución de 21 de octubre de 2020',
 'Resolución CLT/264/2017',
 'Resolución CLT/145/2018',
 'Ley 38/2003',
 'Ley 17/2015',
 'Ley 20/2010',
 'Resolución CLT/198/2021',
 'Resolución de 15 de febrero de 2021',
 'Resolución GAH/380/2016',
 'Reglamento (UE) 2016/679',
 '

In [38]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


['España', 'Europa', 'Cataluña']


In [39]:
#boletin_ASECORP_flat_list

In [40]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

set()

In [41]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 27/2011'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Real decreto 463/2020'}
{'Real decreto 463/2020'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 5/2020'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 17/2015'}
{'Ley 17/2015'}
{'Ley 17/2015'}


In [42]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 27/2011'} LEY 27/2011 ESP
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Real decreto 463/2020'} RES TES/1034/2020 CAT
{'Real decreto 463/2020'} RES TES/973/2020 CAT
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 5/2020'} LEY 5/2020 CAT
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 17/2015'} LEY 17/2015 ESP
{'Ley 17/2015'} LEY 17/2015 ESP
{'Ley 17/2015'} LEY 17/2015 ESP


In [43]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,Tags,Match_ASECORP_BBDD
0,"RESOLUCIÓN EXI/583/2021, de 24 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,En fecha 1 de febrero de 2021 se ha firmado el...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[24/02/2021],[EXI/0583/2021],[21062037],[Agencia Catalana de Cooperación al Desarrollo],[CVE-DOGC-B-21062037-2021],[8357],[05/03/2021],[Disposiciones],"[Ley 19/2014, Resolución EXI/1349/2017, ley 8/...",[LEY 27/2011 ESP]
1,"EDICTO de 25 de febrero de 2021, sobre acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de la Cat...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[25/02/2021],[],[21057021],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21057021-2021],[8357],[05/03/2021],[Disposiciones],"[Ley 29/1998, Real decreto 463/2020, Ley 3/201...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
2,"EDICTO de 26 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"La Comisión de Territorio de Cataluña, en la s...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[26/02/2021],[],[21057014],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21057014-2021],[8357],[05/03/2021],[Disposiciones],"[Ley 39/2015, Ley 29/1998, Ley 5/2020, Ley 3/2...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
3,"EDICTO de 26 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo del ámbit...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[26/02/2021],[],[21057074],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21057074-2021],[8357],[05/03/2021],[Disposiciones],"[Ley 39/2015, Ley 29/1998, Ley 3/2012, Resoluc...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
4,"RESOLUCIÓN TES/585/2021, de 23 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El artículo 3.6 del Decreto 1/2018, de 19 de m...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[23/02/2021],[TES/0585/2021],[21062039],[Agencia de la Vivienda de Cataluña],[CVE-DOGC-B-21062039-2021],[8357],[05/03/2021],[Disposiciones],"[Ley 38/2003, Ley 19/2014, Ley 13/2009, Ley 39...",[]
5,"RESOLUCIÓN CLT/592/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Considerando el Acuerdo del Consejo de Adminis...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[CLT/0592/2021],[21062040],[Oficina de Apoyo a la Iniciativa Cultural],[CVE-DOGC-B-21062040-2021],[8357],[05/03/2021],[Disposiciones],"[Ley 38/2003, Resolución CLT/198/2021, Ley 9/1...",[]
6,"RESOLUCIÓN CLT/576/2021, de 1 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,\nVisto el Acuerdo del Consejo de Administrac...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[01/03/2021],[CLT/0576/2021],[21061067],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21061067-2021],[8357],[05/03/2021],[Disposiciones],"[Resolución CLT/187/2021, Ley 38/2003, Ley 17/...",[LEY 17/2015 ESP]
7,"RESOLUCIÓN CLT/577/2021, de 1 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[01/03/2021],[CLT/0577/2021],[21061068],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21061068-2021],[8357],[05

# Genera Fichero EXCEL de resultados

In [44]:
DOGC_sumarios_final = DOGC_sumarios

In [45]:
DOGC_sumarios_final.dtypes

item_Title            object
item_urlHTML          object
item_urlPDF           object
item_fullText         object
item_urlXML           object
tipo_doc              object
fecha_doc             object
numero_doc            object
numero_control        object
entidad_emisora       object
CVE                   object
numero_DOGC           object
fecha_DOGC            object
seccion_DOGC          object
Tags                  object
Match_ASECORP_BBDD    object
dtype: object

In [46]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [47]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [48]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [49]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [50]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [51]:
DOGC_sumarios_final_CSV['item_id'][0]

'=HIPERVINCULO("https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=894875&type=01&language=es_ES";"CVE-DOGC-B-21062037-2021")'

In [52]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [53]:
DOGC_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,CVE-DOGC-B-21062037-2021,"RESOLUCIÓN EXI/583/2021, de 24 de febrero, por...",05/03/2021,"Ley 19/2014, Resolución EXI/1349/2017, ley 8/2...",LEY 27/2011 ESP
1,CVE-DOGC-B-21057021-2021,"EDICTO de 25 de febrero de 2021, sobre acuerdo...",05/03/2021,"Ley 29/1998, Real decreto 463/2020, Ley 3/2012...","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
2,CVE-DOGC-B-21057014-2021,"EDICTO de 26 de febrero de 2021, sobre un acue...",05/03/2021,"Ley 39/2015, Ley 29/1998, Ley 5/2020, Ley 3/20...","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
3,CVE-DOGC-B-21057074-2021,"EDICTO de 26 de febrero de 2021, sobre un acue...",05/03/2021,"Ley 39/2015, Ley 29/1998, Ley 3/2012, Resoluci...","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
4,CVE-DOGC-B-21062039-2021,"RESOLUCIÓN TES/585/2021, de 23 de febrero, por...",05/03/2021,"Ley 38/2003, Ley 19/2014, Ley 13/2009, Ley 39/...",
5,CVE-DOGC-B-21062040-2021,"RESOLUCIÓN CLT/592/2021, de 2 de marzo, por la...",05/03/2021,"Ley 38/2003, Resolución CLT/198/2021, Ley 9/19...",
6,CVE-DOGC-B-21061067-2021,"RESOLUCIÓN CLT/576/2021, de 1 de marzo, por la...",05/03/2021,"Resolución CLT/187/2021, Ley 38/2003, Ley 17/2...",LEY 17/2015 ESP
7,CVE-DOGC-B-21061068-2021,"RESOLUCIÓN CLT/577/2021, de 1 de marzo, por la...",05/03/2021,"Ley 38/2003, Ley 17/2015, Ley 20/2010, Reglame...",LEY 17/2015 ESP
8,CVE-DOGC-B-21062044-2021,"RESOLUCIÓN CLT/589/2021, de 2 de marzo, por la...",05/03/2021,"Resolución CLT/187/2021, Ley 38/2003, Resoluci...",
9,CVE-DOGC-B-21062045-2021,"RESOLUCIÓN CLT/590/2021, de 2 de marzo, por la...",05/03/2021,"Ley 38/2003, Resolución CLT/413/2018, Resoluci...",


In [54]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 